<a href="https://colab.research.google.com/github/cbedart/EGBSL_IAEA/blob/main/EGBSL_IAEA_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<center><h1>EGBSL - IAES - Part #1</h1></center>**

---

In [ ]:
# @title Loading of prerequisites packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns; sns.set()
from sklearn.decomposition import PCA, KernelPCA
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import datasets
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import clear_output

import plotly.express as px

np.seterr(divide='ignore', invalid='ignore')

!wget https://github.com/cbedart/EGBSL_IAEA/raw/refs/heads/main/computed_desc_GR_tosave.pkl
print("\n\n\033[1mPrerequisites succesfully installed !\033[0m")

In [ ]:
# @title Glucocorticoid ligand dataset
# @markdown Dataset information:
# @markdown - 218 features = Molecular descriptors
# @markdown - Generated from 1703 different compounds
# @markdown - BindingDB_ID = Identifier
# @markdown - IC50/pIC50 = Biological activity value

desc_GR_noNA = pd.read_pickle("computed_desc_GR_tosave.pkl").dropna(axis="columns")
desc_GR = desc_GR_noNA.copy()
desc_GR_noNA

<h1><b><font color='#0E0'> → Explore the dataset to see the different features </font></b></h1>

---


In [ ]:
# @title The very first Principal Component Analysis (PCA) model

X = desc_GR_noNA.iloc[:,3:]

pca_v1 = PCA(n_components=10)
pca_v1_result = pca_v1.fit_transform(X)
principal_components = pca_v1.components_

plt.figure(figsize=(8, 6))
plt.scatter(pca_v1_result[:, 0], pca_v1_result[:, 1])
plt.title('PCA of Descriptor Data')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.show()

<h1><b><font color='#0E0'> → Is there something strange with this PCA?</font></b></h1>

---


In [ ]:
# @title Scaling & normalization of the dataset

scaler_temp = preprocessing.StandardScaler()
norm_temp = pd.DataFrame(preprocessing.normalize(scaler_temp.fit_transform(desc_GR_noNA.iloc[:,3:])))
norm_temp.columns = desc_GR_noNA.iloc[:,3:].columns

desc_GR_normalized = pd.concat([desc_GR_noNA.iloc[:,:3], norm_temp], axis="columns")

desc_GR_normalized_noNA = desc_GR_normalized.dropna(axis="columns")
desc_GR_normalized_noNA

In [ ]:
# @title Second PCA model

X = desc_GR_normalized_noNA.iloc[:,3:]

pca_v2 = PCA(n_components=30)
pca_v2_result = pca_v2.fit_transform(X)
principal_components = pca_v2.components_


plt.figure(figsize=(8, 6))
plt.scatter(pca_v2_result[:, 0], pca_v2_result[:, 1])
plt.title('PCA of Descriptor Data')
plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v2.explained_variance_[0]))
plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v2.explained_variance_[1]))
plt.grid(True)
plt.show()

print("Explained variances of the 30 first principal components:")
print(pca_v2.explained_variance_)

<h1><b><font color='#0E0'> → Is this better?</font></b></h1>

---


In [ ]:
# @title In 3D

fig2 = px.scatter_3d(x=pca_v2_result[:, 0], y=pca_v2_result[:, 1], z=pca_v2_result[:, 2], width=600, height=600)
fig2.update_traces(marker_size = 2)
fig2.show()

In [ ]:
#@title Same PCA but with custom color parameters

descriptor_picker = widgets.Dropdown(options=desc_GR_normalized_noNA.columns[1:], value='MolWt')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
outs = widgets.Output()

def generation_PCA_fig_v2(x):
  with outs:
    clear_output()

    plt.figure(figsize=(8, 6))
    # plt.subplot(121)

    a = plt.scatter(pca_v2_result[:,0], pca_v2_result[:,1], c = desc_GR[descriptor_picker.value], cmap="turbo")
    plt.colorbar(a)
    plt.title('PCA of Descriptor Data')
    plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v2.explained_variance_[0]))
    plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v2.explained_variance_[1]))
    plt.grid(True)
    plt.show()
    # plt.subplot(122, projection='3d')
    # b = plt.scatter(pca_v2_result[:,0], pca_v2_result[:,1], pca_v2_result[:,2], c = desc_GR[descriptor_picker.value], cmap="turbo")
    # plt.colorbar(b)
    # plt.title('PCA of Descriptor Data')
    # plt.xlabel('Principal Component 1')
    # plt.ylabel('Principal Component 2')
    # plt.grid(True)
    # plt.show()

display(descriptor_picker)
display(outs)
generation_PCA_fig_v2(descriptor_picker.value)
descriptor_picker.observe(generation_PCA_fig_v2, names='value')

In [ ]:
#@title Feature importance in PC1 and PC2

feature_names = desc_GR_normalized_noNA.iloc[:, 3:].columns

pc1_loadings = pd.Series(np.abs(pca_v2.components_[0]), index=feature_names)
top_30_pc1 = pc1_loadings.sort_values(ascending=False).head(30)
plt.figure(figsize=(10, 8))
sns.barplot(x=top_30_pc1.values, y=top_30_pc1.index)
plt.title('Top 30 Features in Principal Component 1')
plt.xlabel('Absolute Loading Value')
plt.ylabel('Feature Name')
plt.tight_layout()
plt.show()

pc2_loadings = pd.Series(np.abs(pca_v2.components_[1]), index=feature_names)
top_30_pc2 = pc2_loadings.sort_values(ascending=False).head(30)
plt.figure(figsize=(10, 8))
sns.barplot(x=top_30_pc2.values, y=top_30_pc2.index)
plt.title('Top 30 Features in Principal Component 2')
plt.xlabel('Absolute Loading Value')
plt.ylabel('Feature Name')
plt.tight_layout()
plt.show()


<h1><b><font color='#0E0'> → Do any features seem correlated to you?</font></b></h1>

---


In [ ]:
# @title Removal of highly correlated features

def elimcor_sansY(X, s=0.95):
    correl = np.corrcoef(X, rowvar=False)
    stop = False
    possetap = list(range(X.shape[1]))
    groupes = {i: [i] for i in range(X.shape[1])}

    while not stop:
        # Regroup variables for which |corr| > threshold
        gplist = {}
        possglob = list(range(correl.shape[0]))
        for i in range(correl.shape[0]):
            poss = possglob[:i] + possglob[i+1:]
            gplist[i] = [i] + [poss[j] for j in range(len(poss)) if abs(correl[i, poss[j]]) > s]

        # Sort groups from largest to smallest
        gplisteff = {k: len(v) for k, v in gplist.items()}
        if any(val > 1 for val in gplisteff.values()):
            gplistfin = {k: v for k, v in gplist.items() if gplisteff[k] > 1}
            gplistuniq = [k for k, v in gplisteff.items() if v == 1]
            gpsel = []
            for i in gplistfin.values():
                selloc = min(i)
                gploc = groupes[possetap[selloc]]
                for j in i:
                    gploc.extend(groupes[possetap[j]])
                groupes[possetap[selloc]] = list(set(gploc))
                gpsel.append(selloc)
            possetap = [val for idx, val in enumerate(possetap) if idx in gplistuniq or val in set(gpsel)]
            correl = np.corrcoef(X.iloc[:,possetap], rowvar=False)
        else:
            stop = True

    return {"possetap": possetap, "groups": list(groupes.values())}

################################################################################

X = desc_GR_normalized_noNA.iloc[:,3:]
elimcor = elimcor_sansY(X, 0.9)
desc_GR_nocorr = pd.concat([desc_GR_normalized_noNA.iloc[:,:3], X.iloc[:,elimcor["possetap"]]], axis="columns")
desc_GR_nocorr

In [ ]:
#@title Third PCA model

X = desc_GR_nocorr.iloc[:,3:]

pca_v3 = PCA(n_components=30)
pca_v3_result = pca_v3.fit_transform(X)
principal_components = pca_v3.components_

descriptor_picker2 = widgets.Dropdown(options=desc_GR_nocorr.columns[1:], value='Steroidal')

outs = widgets.Output()

def generation_PCA_fig_v3(x):
  with outs:
    clear_output()
    plt.figure(figsize=(8, 6))
    a = plt.scatter(pca_v3_result[:,0], pca_v3_result[:,1], c = desc_GR[descriptor_picker2.value], cmap="turbo")
    plt.colorbar(a)
    plt.title('PCA of Descriptor Data')
    plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[0]))
    plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[1]))
    plt.grid(True)
    plt.show()

display(descriptor_picker2)
display(outs)
generation_PCA_fig_v3(descriptor_picker2.value)
descriptor_picker2.observe(generation_PCA_fig_v3, names='value')

print("Explained variances of the 30 first principal components:")
print(pca_v3.explained_variance_)

<h1><b><font color='#0E0'> → Can we cluster the steroidal feature to remove them?</font></b></h1>

---

---

---

In [ ]:
# @title Where are the steroidal compounds?

fig3 = px.scatter_3d(x=pca_v3_result[:, 0], y=pca_v3_result[:, 1], z=pca_v3_result[:, 2], width=600, height=600, color_continuous_scale=["red", "blue"], color=desc_GR["Steroidal"], labels={"color": "Steroidal"})
fig3.update_traces(marker_size = 2)
fig3.show()

In [ ]:
# @title K-means clustering to generate X groups

X = desc_GR_nocorr.iloc[:,3:]

choice_nb_clusters = widgets.Dropdown(options=list(range(2,10)), value=2)
outs_kmeans = widgets.Output()

def on_nb_change(change):
  with outs_kmeans:
    clear_output()
    kmeans = KMeans(n_clusters=choice_nb_clusters.value, n_init=29, random_state=0)
    kmeans.fit(X)
    cluster_labels = kmeans.labels_
    cluster_labels
    cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
    cluster_centers.columns = X.columns
    cluster_centers_pca = pca_v3.transform(cluster_centers)
    plt.figure(figsize=(6.3, 6))
    plt.scatter(pca_v3_result[:, 0], pca_v3_result[:, 1], c=cluster_labels, cmap="Accent")
    plt.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1], marker='o', c='red', s=20, label='Centroids')
        # Add cluster ID next to each centroid
    for i, txt in enumerate(cluster_centers.index):
        plt.annotate(txt, (cluster_centers_pca[i, 0], cluster_centers_pca[i, 1]), textcoords="offset points", xytext=(0,10), ha='center', fontweight="bold")

    plt.title('KMeans Clustering with {} clusters'.format(choice_nb_clusters.value))
    plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[0]))
    plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[1]))
    plt.legend()
    plt.show()

    plt.figure(figsize=(8, 6))
    a = plt.scatter(pca_v3_result[:,0], pca_v3_result[:,1], c = desc_GR["Steroidal"], cmap="turbo")
    plt.colorbar(a)
    plt.title('PCA of Descriptor Data')
    plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[0]))
    plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v3.explained_variance_[1]))
    plt.grid(True)
    plt.show()

    desc_GR_clustered = pd.concat([desc_GR_nocorr.iloc[:,:3], pd.DataFrame(cluster_labels, columns=["Cluster"]), desc_GR_nocorr.iloc[:,3:]], axis = "columns")
    desc_GR_clustered['Steroidal'] = desc_GR_clustered['Steroidal'].apply(lambda x: 'Steroidal' if x > 0 else 'Non steroidal')
    print(desc_GR_clustered.groupby("Cluster")["Steroidal"].value_counts())

    desc_GR_clustered = pd.concat([desc_GR_nocorr.iloc[:,:3], pd.DataFrame(cluster_labels, columns=["Cluster"]), desc_GR_nocorr.iloc[:,3:]], axis = "columns")
    display(desc_GR_clustered)

display(choice_nb_clusters)
display(outs_kmeans)

on_nb_change(2)
choice_nb_clusters.observe(on_nb_change, names='value')


<h1><b><font color='#0E0'> → How many clusters we can divide the subset into to efficiently remove the steroidal compounds?</font></b></h1>

---

In [ ]:
# @title With 3 clusters, by removing the right cluster that includes the steroids
# @ markdown Data have been re-normalized

X = desc_GR_nocorr.iloc[:,3:]
kmeans = KMeans(n_clusters=3, n_init=29, random_state=0)
kmeans.fit(X)
cluster_labels = kmeans.labels_
desc_GR_clustered = pd.concat([desc_GR_nocorr.iloc[:,:3], pd.DataFrame(cluster_labels, columns=["Cluster"]), desc_GR_nocorr.iloc[:,3:]], axis = "columns")

desc_GR_clustered_filtered = desc_GR_nocorr.loc[desc_GR_clustered["Cluster"] != desc_GR_clustered.loc[1700,"Cluster"]]
desc_GR_clustered_filtered = desc_GR_clustered_filtered.dropna(axis="columns")
desc_GR_clustered_filtered.index = range(len(desc_GR_clustered_filtered))

# desc_GR_clustered_normalized = pd.concat([desc_GR_clustered_filtered.iloc[:,:4], (desc_GR_clustered_filtered.iloc[:,4:] - desc_GR_clustered_filtered.iloc[:,4:].mean()) / desc_GR_clustered_filtered.iloc[:,4:].std()], axis="columns")

scaler_temp = preprocessing.StandardScaler()
norm_temp = pd.DataFrame(preprocessing.normalize(scaler_temp.fit_transform(desc_GR_clustered_filtered.iloc[:,4:])))
norm_temp.columns = desc_GR_clustered_filtered.iloc[:,4:].columns

desc_GR_clustered_normalized = pd.concat([desc_GR_clustered_filtered.iloc[:,:4], norm_temp], axis="columns")
desc_GR_clustered_normalized_noNA = desc_GR_clustered_normalized.dropna(axis="columns")
desc_GR_clustered_normalized_noNA



X = desc_GR_clustered_normalized_noNA.iloc[:,3:]

pca_v4 = PCA(n_components=30)
pca_v4_result = pca_v4.fit_transform(X)
principal_components = pca_v4.components_

descriptor_picker2 = widgets.Dropdown(options=desc_GR_clustered_normalized_noNA.columns[1:], value='Steroidal')

outs = widgets.Output()

def generation_PCA_fig_v4(x):
  with outs:
    clear_output()
    plt.figure(figsize=(8, 6))
    a = plt.scatter(pca_v4_result[:,0], pca_v4_result[:,1], c = desc_GR_clustered_normalized_noNA[descriptor_picker2.value], cmap="turbo")
    plt.colorbar(a)
    plt.title('PCA of Descriptor Data')
    plt.xlabel('Principal Component 1 - {:.2f}% of explained variance'.format(100*pca_v4.explained_variance_[0]))
    plt.ylabel('Principal Component 2 - {:.2f}% of explained variance'.format(100*pca_v4.explained_variance_[1]))
    plt.grid(True)
    plt.show()

display(descriptor_picker2)
display(outs)
generation_PCA_fig_v4(descriptor_picker2.value)
descriptor_picker2.observe(generation_PCA_fig_v4, names='value')

print("Explained variances of the 30 first principal components:")
print(pca_v4.explained_variance_)


desc_GR_clustered_normalized_noNA

In [ ]:
#@title How many compounds to keep to explain 95% of the total variance?

X = desc_GR_clustered_normalized_noNA.iloc[:,3:]
pca_v5 = PCA(n_components=60)
pca_v5_result = pca_v5.fit_transform(X)
principal_components = pca_v5.components_

cumulative_variance = np.cumsum(pca_v5.explained_variance_ratio_)

plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', markersize=3)
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
plt.legend()
plt.grid(True)
plt.show()

n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"95% cumulative explained variances using the \033[1m{n_components_95}\033[0m first principal components:")

######################

desc_GR_PCA_53c_temp = pd.DataFrame(pca_v5_result).iloc[:,0:n_components_95]
desc_GR_PCA_53c = pd.concat([desc_GR_clustered_normalized_noNA.iloc[:,0:3], desc_GR_PCA_53c_temp], axis=1)
desc_GR_PCA_53c


---

---

---

In [ ]:
#@title Install tSNE and UMAP packages

!pip install umap-learn
from sklearn.manifold import TSNE
import umap

print("Installed!")

In [ ]:
#@title UMAP vs tSNE vs UMAP using the final dataset ...

X = desc_GR_clustered_normalized_noNA.iloc[:, 3:]

if 'tsne_result' not in locals() and 'tsne_result' not in globals():
    tsne = TSNE(n_components=2, random_state=42, n_iter=300) # Reduced iterations for faster execution
    tsne_result = tsne.fit_transform(X)

if 'umap_result' not in locals() and 'umap_result' not in globals():
  umap_reducer = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.1, random_state=42)
  umap_result = umap_reducer.fit_transform(X)

# Create a DataFrame for plotting
plot_data = pd.DataFrame({'PCA_1': pca_v4_result[:, 0],
                          'PCA_2': pca_v4_result[:, 1],
                          'tSNE_1': tsne_result[:, 0],
                          'tSNE_2': tsne_result[:, 1],
                          'UMAP_1': umap_result[:, 0],
                          'UMAP_2': umap_result[:, 1],
                          'Steroidal': desc_GR_clustered_normalized_noNA['Steroidal'],
                          'BindingDB_ID': desc_GR_clustered_normalized_noNA['BindingDB_ID'],
                          'IC50': desc_GR_clustered_normalized_noNA['IC50']
                         })

# Plotting
plt.figure(figsize=(18, 6))

# PCA plot
plt.subplot(1, 3, 1)
sns.scatterplot(x='PCA_1', y='PCA_2', data=plot_data, palette='viridis', s=20)
plt.title('PCA')
plt.xlabel(f'Principal Component 1 - {100*pca_v3.explained_variance_[0]:.2f}%')
plt.ylabel(f'Principal Component 2 - {100*pca_v3.explained_variance_[1]:.2f}%')
plt.grid(True)

# t-SNE plot
plt.subplot(1, 3, 2)
sns.scatterplot(x='tSNE_1', y='tSNE_2', data=plot_data, palette='viridis', s=20)
plt.title('t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)

# UMAP plot
plt.subplot(1, 3, 3)
sns.scatterplot(x='UMAP_1', y='UMAP_2', data=plot_data, palette='viridis', s=20)
plt.title('UMAP')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
#@title ... and UMAP vs tSNE vs UMAP using the dataset before removing the steroidal compounds

X = desc_GR_nocorr.iloc[:, 3:]

if 'tsne_result2' not in locals() and 'tsne_result2' not in globals():
    tsne = TSNE(n_components=2, random_state=42, n_iter=300)
    tsne_result2 = tsne.fit_transform(X)

if 'umap_result2' not in locals() and 'umap_result2' not in globals():
  umap_reducer = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.1, random_state=42)
  umap_result2 = umap_reducer.fit_transform(X)

# Create a DataFrame for plotting
plot_data = pd.DataFrame({'PCA_1': pca_v3_result[:, 0],
                          'PCA_2': pca_v3_result[:, 1],
                          'tSNE_1': tsne_result2[:, 0],
                          'tSNE_2': tsne_result2[:, 1],
                          'UMAP_1': umap_result2[:, 0],
                          'UMAP_2': umap_result2[:, 1],
                          'Steroidal': desc_GR_nocorr['Steroidal'].apply(lambda x: 'Steroidal' if x > 0 else 'Non steroidal'),
                          'BindingDB_ID': desc_GR_nocorr['BindingDB_ID'],
                          'IC50': desc_GR_nocorr['IC50']
                         })

# Plotting
plt.figure(figsize=(18, 6))

# PCA plot
plt.subplot(1, 3, 1)
sns.scatterplot(x='PCA_1', y='PCA_2', hue="Steroidal", data=plot_data, s=20)
plt.title('PCA')
plt.xlabel(f'Principal Component 1 - {100*pca_v3.explained_variance_[0]:.2f}%')
plt.ylabel(f'Principal Component 2 - {100*pca_v3.explained_variance_[1]:.2f}%')
plt.grid(True)

# t-SNE plot
plt.subplot(1, 3, 2)
sns.scatterplot(x='tSNE_1', y='tSNE_2', hue="Steroidal", data=plot_data, s=20)
plt.title('t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True)

# UMAP plot
plt.subplot(1, 3, 3)
sns.scatterplot(x='UMAP_1', y='UMAP_2', hue="Steroidal", data=plot_data, s=20)
plt.title('UMAP')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.grid(True)

plt.tight_layout()
plt.show()